# Goal

The objective of this notebook is to demonstrate and visualize the computation of the pareto frontier.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd


from mlos.OptimizerEvaluationTools.ObjectiveFunctionFactory import ObjectiveFunctionFactory
from mlos.OptimizerEvaluationTools.SyntheticFunctions.Hypersphere import Hypersphere

from mlos.Optimizers.BayesianOptimizerFactory import BayesianOptimizerFactory, bayesian_optimizer_config_store
from mlos.Optimizers.OptimizationProblem import OptimizationProblem, Objective
from mlos.Optimizers.ParetoFrontier import ParetoFrontier

from mlos.Spaces import ContinuousDimension, DiscreteDimension, Point, SimpleHypergrid

hypersphere_radius = 10

objective_function_config = Point(
    implementation=Hypersphere.__name__,
    hypersphere_config=Point(
        num_objectives=3,
        minimize='none',
        radius=hypersphere_radius
    )
)

objective_function = ObjectiveFunctionFactory.create_objective_function(objective_function_config=objective_function_config)
optimization_problem = objective_function.default_optimization_problem
optimizer = BayesianOptimizerFactory().create_local_optimizer(
    optimization_problem=optimization_problem
)

01/16/2021 00:51:06 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizerFactory.py:  37 -    create_local_optimizer() ] Optimizer config not specified. Using default.


In [3]:
optimizer.optimizer_config

{
  "surrogate_model_implementation": "HomogeneousRandomForestRegressionModel",
  "experiment_designer_implementation": "ExperimentDesigner",
  "min_samples_required_for_guided_design_of_experiments": 10,
  "homogeneous_random_forest_regression_model_config.n_estimators": 10,
  "homogeneous_random_forest_regression_model_config.features_fraction_per_estimator": 1,
  "homogeneous_random_forest_regression_model_config.samples_fraction_per_estimator": 1,
  "homogeneous_random_forest_regression_model_config.regressor_implementation": "DecisionTreeRegressionModel",
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.criterion": "mse",
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.splitter": "best",
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.max_depth": 0,
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.min_samples_spli

In [4]:
num_iterations = 100
for i in range(num_iterations):
    print(i)
    config = optimizer.suggest()
    value = objective_function.evaluate_point(config)
    optimizer.register(config.to_dataframe(), value.to_dataframe())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36


KeyboardInterrupt: 

In [5]:
# Now let's plot the existing pareto frontier, along with some predictions.
#
params_df, objectives_df, context_df = optimizer.get_all_observations()

# Let's create pareto frontiers over time so that we can plot them.
#
pareto_dfs_over_time = [ParetoFrontier.compute_pareto(optimization_problem, objectives_df[:i]) for i in objectives_df.index]
objectives_over_time = [objectives_df[:i] for i in objectives_df.index]

In [13]:
import plotly.graph_objects as go


# Create figure
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=objectives_over_time[0]['y0'],
            y=objectives_over_time[0]['y1'],
            z=objectives_over_time[0]['y2'],
            mode="markers",
            marker=dict(color="blue", size=10),
            name='observed objectives'
        ),
        go.Scatter3d(
            x=pareto_dfs_over_time[0]['y0'],
            y=pareto_dfs_over_time[0]['y1'],
            z=objectives_over_time[0]['y2'],
            name='pareto-ish'
            #mode="markers",
            #marker=dict(color="red", size=10)
        )
    ],
    layout=go.Layout(
        scene = dict(
            xaxis=dict(nticks=4, range=[0,10]),
            yaxis=dict(nticks=4, range=[0,10]),
            zaxis=dict(nticks=4, range=[0,10]),
        ),
        title_text="Pareto over time",
        hovermode="closest",
        updatemenus=[dict(type="buttons", buttons=[dict(label="Play", method="animate", args=[None])])],
        width=600,
        height=600
    ),
    frames=[
        go.Frame(
            data=[
                go.Scatter3d(
                    x=objectives_over_time[i]['y0'],
                    y=objectives_over_time[i]['y1'],
                    z=objectives_over_time[i]['y2'],
                    mode="markers",
                    marker=dict(color="blue", size=10),
                    name='observed objectives'
                ),
                go.Scatter3d(
                    x=pareto_dfs_over_time[i]['y0'],
                    y=pareto_dfs_over_time[i]['y1'],
                    z=pareto_dfs_over_time[i]['y2'],
                    name='pareto-ish'
                    #mode="markers",
                    #marker=dict(color="red", size=10)
                )
            ]
        )
        for i in objectives_df.index
    ]
)

camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=10, y=10, z=10)
)

#fig.update_layout(scene_camera=camera, title='Pareto over time')

fig.show()